In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

def con(row):
    if row['Sex']=='male'and row['Age']>=18:
        row['Sex']=2 #man
    elif row['Sex']=='male':
        row['Sex']=1 #boy
    else:
        row['Sex']=0 #female
    return row
def con2(row):
    if row['Embarked']=='S':
        row['Embarked']=1
    elif row['Embarked']=='Q':
        row['Embarked']=2
    else:
        row['Embarked']=0
    return row

train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
m=test_data['Fare'].mean()
test_data['Fare']=test_data['Fare'].fillna(m)
test_data['Age']=test_data['Age'].fillna(24)
train_data['Age']=train_data['Age'].fillna(24)
train_data=train_data.apply(con2,axis=1)
test_data=test_data.apply(con2,axis=1)

X=train_data.drop('Survived',axis=1)
y=train_data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model1=Pipeline([('scaler',MinMaxScaler()),('knn',KNeighborsClassifier(n_neighbors=4))])
model2=LogisticRegression(solver='liblinear',penalty='l1',C=1)
model3=Pipeline([('scaler',StandardScaler()),('svc',SVC(kernel='rbf'))])
model4=DecisionTreeClassifier(splitter='random',max_depth=3)
train_data['type']='train'
test_data['type']='test'
all = pd.concat([train_data, test_data], sort=False).reset_index(drop=True)
all_corr=all.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
all['Age'] = all.groupby(['Pclass', 'SibSp'])['Age'].apply(lambda x: x.fillna(x.median()))
all['Fare'] = all.groupby(['Pclass', 'Parch', 'SibSp'])['Fare'].apply(lambda x: x.fillna(x.median()))

all['Cabin'] = all['Cabin'].fillna('N')
all['Embarked'] = all['Embarked'].fillna(1)
all['Name']=all['Name'].str.replace(r'(','')
all['Title'] = all['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
all['IsMarried'] = np.where(all['Title'] == 'Mrs', 1, 0)
all['Family']=all['Name'].str.split(', ', expand=True)[0]

all=all.drop(['Name'],axis=1)
all['FamilySize'] = all['SibSp'] + all['Parch'] + 1
family_map = {1: 'Alone', 2: 'Small', 3: 'Small', 4: 'Small', 5: 'Medium', 6: 'Medium', 7: 'Large', 8: 'Large', 11: 'Large'}
all['FamilySizeGrouped'] = all['FamilySize'].map(family_map)
all['Ticket_count'] = all.Ticket.apply(lambda x: all[all['Ticket']==x].shape[0])
all=all.apply(con,axis=1)
list1 = all[(all['Title'] != 'Mr') & (all['Survived'] == 0) ]['Ticket'].tolist() # Female and child no survide.
list2 = all[(all['Title'] == 'Mr') & (all['Survived'] == 1) ]['Ticket'].tolist() # Man survive.
all['Ticket_with_FC_dead'] = 0
all['Ticket_with_M_alive'] = 0
all.loc[all['Ticket'].isin(list1), 'Ticket_with_FC_dead'] = 1
all.loc[all['Ticket'].isin(list2), 'Ticket_with_M_alive'] = 1
all.drop(['Ticket'], inplace=True, axis=1)
all['Deck'] = all['Cabin'].apply(lambda x: x[0])
all['Deck'] = all['Deck'].replace(['T'], 'A')
all.drop(['Cabin'], inplace=True, axis=1)
all.loc[all['Deck'] == 'A', 'Deck'] = 0
all.loc[all['Deck'] == 'B', 'Deck'] = 1
all.loc[all['Deck'] == 'C', 'Deck'] = 2
all.loc[all['Deck'] == 'D', 'Deck'] = 3
all.loc[all['Deck'] == 'E', 'Deck'] = 4
all.loc[all['Deck'] == 'F', 'Deck'] = 5
all.loc[all['Deck'] == 'G', 'Deck'] = 6
all.loc[all['Deck'] == 'N', 'Deck'] = 7
all['Title'] = all['Title'].replace(['Miss', 'Mrs','Ms', 'Mlle', 'Lady', 'Mme', 'the Countess', 'Dona'], 'Miss/Mrs/Ms')
all['Title'] = all['Title'].replace(['Dr', 'Col', 'Major', 'Jonkheer', 'Capt', 'Sir', 'Don', 'Rev'], 'Dr/Military/Noble/Clergy')
all.loc[all['Title'] == 'Mr', 'Title'] = 0
all.loc[all['Title'] == 'Miss/Mrs/Ms', 'Title'] = 1
all.loc[all['Title'] == 'Master', 'Title'] = 2
all.loc[all['Title'] == 'Dr/Military/Noble/Clergy', 'Title'] = 3
list1 = all[(all['Title'] != 0) & (all['Survived'] == 0) ]['Family'].tolist() # Female and child no survide.
list2 = all[(all['Title'] == 0) & (all['Survived'] == 1) ]['Family'].tolist() # Man survive.
all['Family_with_FC_dead'] = 0
all['Family_with_M_alive'] = 0
all.loc[all['Family'].isin(list1), 'Family_with_FC_dead'] = 1
all.loc[all['Family'].isin(list2), 'Family_with_M_alive'] = 1
all.drop(['Family'], inplace=True, axis=1)
all.loc[all['FamilySizeGrouped'] == 'Alone', 'FamilySizeGrouped'] = 0
all.loc[all['FamilySizeGrouped'] == 'Small', 'FamilySizeGrouped'] = 1
all.loc[all['FamilySizeGrouped'] == 'Medium', 'FamilySizeGrouped'] = 2
all.loc[all['FamilySizeGrouped'] == 'Large', 'FamilySizeGrouped'] = 3

all=all.drop(["SibSp","Parch","FamilySize","IsMarried"],axis=1)
train=all[all['type']=='train']
test=all[all['type']=='test']

train=train.drop('type',axis=1)
test=test.drop(["type","Survived"],axis=1)
X=train.drop(["PassengerId","Survived"],axis=1)
y=train["Survived"]


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


model1=Pipeline([('scaler',MinMaxScaler()),('knn',KNeighborsClassifier(n_neighbors=3))])
model2=LogisticRegression(solver='newton-cg',max_iter=30)
model3=Pipeline([('scaler',StandardScaler()),('svc',SVC(kernel='rbf'))])
model4=DecisionTreeClassifier(max_depth=3,splitter='random')

test_data1=test.drop("PassengerId",axis=1)
model=model4
model.fit(X,y)

predictions = model.predict(test_data1)
predictions=predictions.astype(int)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
